In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [71]:
# url = 'https://www.nldc.evn.vn/api/services/app/Pages/GetChartNguonDien'
# #get data from url
# response = requests.get(url)
# #convert data to json
# data = response.json()
# #convert data to pandas dataframe
# df = pd.DataFrame(data['result']['data']['nguonDiens'])
# df

# 1.SCS

In [72]:
SCS = pd.DataFrame()
def get_SCS(i):
    url = 'https://ecargo.scsc.vn/General/Tonnage/Public'
    payload = {'year':i}
    response = requests.post(url, data=payload)
    data = response.json()
    df = pd.DataFrame(data)
    return df
for i in range(2017,2025):
    x = get_SCS(i)
    SCS = pd.concat([SCS,x])

In [73]:
SCS.to_excel('C:/Users/hoang.nguyennhat/Desktop/IR/SCS.xlsx')

# 2. TLG

In [74]:
def get_TLG_VI(x):
    response = requests.get(x)
    soup = BeautifulSoup(response.text, 'html.parser')
    titles = []
    pdfs = []
    for link in soup.find_all('a'):
        if 'nhà đầu tư' in link.text.lower():
            titles.append(link.text)
            if link.get('href').endswith('.pdf'):
                pdfs.append(link.get('href'))
    #create a dataframe from titles and pdfs
    df = pd.DataFrame({'Title':titles, 'PDF':pdfs})
    return df

def get_TLG_EN(x):
    response = requests.get(x)
    soup = BeautifulSoup(response.text, 'html.parser')
    titles = []
    pdfs = []
    for link in soup.find_all('a'):
        if 'newsletter' in link.text.lower():
            titles.append(link.text)
            if link.get('href').endswith('.pdf'):
                pdfs.append(link.get('href'))
    #create a dataframe from titles and pdfs
    df = pd.DataFrame({'Title':titles, 'PDF':pdfs})
    return df



In [75]:
TLG_VI = pd.DataFrame()
TLG_EN = pd.DataFrame()
for i in range(1,19):
    url_VI = 'https://thienlonggroup.com/quan-he-co-dong/tai-lieu-khac?page='+str(i)
    url_EN = 'https://thienlonggroup.com/en/quan-he-co-dong/tai-lieu-khac?page='+str(i)
    #use get_TLG function to get data from url and append to dataframe TLG
    x = get_TLG_VI(url_VI)
    y = get_TLG_EN(url_EN)
    TLG_VI = pd.concat([TLG_VI,x])
    TLG_EN = pd.concat([TLG_EN,y])
#save pdf from PDF to C:/Users/hoang.nguyennhat/Desktop/IR/TLG with Title as file name
for i in range(len(TLG_VI)):
    response = requests.get(TLG_VI['PDF'].iloc[i])
    with open('C:/Users/hoang.nguyennhat/Desktop/IR/TLG/VI/'+TLG_VI['Title'].iloc[i]+'.pdf', 'wb') as f:
        f.write(response.content)
for i in range(len(TLG_EN)):
    response = requests.get(TLG_EN['PDF'].iloc[i])
    with open('C:/Users/hoang.nguyennhat/Desktop/IR/TLG/EN/'+TLG_EN['Title'].iloc[i]+'.pdf', 'wb') as f:
        f.write(response.content)

KeyboardInterrupt: 

In [76]:
#get all the filename from C:/Users/hoang.nguyennhat/Desktop/IR/TLG/EN/
import os
files = os.listdir('C:/Users/hoang.nguyennhat/Desktop/IR/TLG/EN/')
#select the string before - of the filename, remove blank space
a = [x.split('-')[0].strip() for x in files]
#remove .pdf from the string
a = [x.replace('.pdf','') for x in a]
#convert files to format YYYY-mm, for example 'August 2021' to '2021-08'
a = [pd.to_datetime(x).strftime('%Y-%m') for x in a]


KeyboardInterrupt: 

In [ ]:
#replace the file in files with list a
for i in range(len(files)):
    os.rename('C:/Users/hoang.nguyennhat/Desktop/IR/TLG/EN/'+files[i],'C:/Users/hoang.nguyennhat/Desktop/IR/TLG/EN/'+a[i]+'.pdf')

# 3. HDG

In [2]:
list_id = []
list_file = []
url = 'https://hado.com.vn/Ajax/Home/LoadContentShareHolders'
def get_id_HDG(i):
    data = {'p': i,'id': 19}
    response = requests.post(url, data=data)
    soup = BeautifulSoup(response.text, 'html.parser')
    #find table in soup
    table = soup.find('table')
    #convert table to dataframe
    df = pd.read_html(str(table))[0]
    #find javascript void in soup
    for link in soup.find_all('a'):
        #select the string include void and get the number in the ()
        if 'javascript:void' in link.get('href'):
            
        #add it into list_id
            list_id.append(link.get('href').split('(')[1].split(')')[0])
            list_file.append(link.text)
        #create the dataframe from list_id and list_file
    df = pd.DataFrame({'ID':list_id, 'File':list_file})
    return df

In [3]:
def get_pdf_HDG(id):
    url = 'https://hado.com.vn/Ajax/Content/ViewDetail?id='+str(id)
    base_url = 'https://hado.com.vn'
    response = requests.get(url)
    #response json
    data = response.json()
    #find href in data
    soup = BeautifulSoup(data['content'], 'html.parser')
    #find all href
    for link in soup.find_all('a'):
        a = (base_url+link.get('href'))
    return a

In [4]:
df = pd.DataFrame()
for i in range(1,5):
    x = get_id_HDG(i)
    df = pd.concat([df,x])

In [17]:
df['PDF'] = df['ID'].apply(get_pdf_HDG)

In [34]:
#use tnrange to create progress bar
from tqdm import tnrange
#replace / with - in File
df['File'] = df['File'].str.replace('/','-')
#save pdf to C:/Users/hoang.nguyennhat/Desktop/IR/HDG
for i in tnrange(len(df)):
    response = requests.get(df['PDF'].iloc[i])
    with open('C:/Users/hoang.nguyennhat/Desktop/IR/HDG/'+str(df['File'].iloc[i])+'.pdf', 'wb') as f:
        f.write(response.content) 

C:\Users\hoang.nguyennhat\AppData\Local\Temp\ipykernel_29068\485858271.py:6: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(df)):


  0%|          | 0/93 [00:00<?, ?it/s]

# 4. PGV

In [99]:
base_url = 'https://www.genco3.com'
def get_pdf_PGV(x):
    response = requests.get(x)
    soup = BeautifulSoup(response.text, 'html.parser')
    list_file = []
    list_name = []
    #file all div class = 'file' in soup
    for link in soup.find_all('div', class_='sn-new-item'):
    #find p class = 'title'
        name = link.find('p', class_='title').text
        #beautify the name
        name = name.replace('\n','').strip()
        list_name.append(name)
        #get pdf link from a tag
        pdf = link.find('a').get('href')
        pdf = base_url+pdf
        list_file.append(pdf)
    #convert list_name and list_file to dataframe
    df = pd.DataFrame({'Name':list_name, 'PDF':list_file})
    #find name include BẢN TIN in Name
    df = df[df['Name'].str.contains('BẢN TIN')]
    #replace / with - in Name
    df['Name'] = df['Name'].str.replace('/','-')
    return df

In [101]:
url = 'https://www.genco3.com/quan-he-nha-dau-tu/ban-tin-nha-dau-tu.html?y=2023'
get_pdf_PGV(url)
#save pdf to C:/Users/hoang.nguyennhat/Desktop/IR/PGV
for i in tnrange(len(df)):
    response = requests.get(df['PDF'].iloc[i])
    with open('C:/Users/hoang.nguyennhat/Desktop/IR/PGV/'+str(df['Name'].iloc[i])+'.pdf', 'wb') as f:
        f.write(response.content)

C:\Users\hoang.nguyennhat\AppData\Local\Temp\ipykernel_29068\1263607968.py:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(len(df)):


  0%|          | 0/88 [00:00<?, ?it/s]

In [ ]:
#rename the file in C:/Users/hoang.nguyennhat/Desktop/IR/PGV to format YYYY-mm, for example 'BẢN TIN NHÀ ĐẦU TƯ THÁNG 01 NĂM 2020' to '2020-01-PGV'
files = os.listdir('C:/Users/hoang.nguyennhat/Desktop/IR/PGV/')
a = [x.split('NĂM ')[1].split(' ')[0] for x in files]
a = [pd.to_datetime(x).strftime('%Y-%m') for x in a]
for i in range(len(files)):
    os.rename('C:/Users/hoang.nguyennhat/Desktop/IR/PGV/'+files[i],'C:/Users/hoang.nguyennhat/Desktop/IR/PGV/'+a[i]+'-PGV.pdf')
    